In [1]:
import pandas as pd
import polars as pl
import numpy as np

/Users/silashayes/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [14]:
cols = ['VENDOR_UEI', 'VENDOR_ADDRESS_ZIP_CODE', #silas
        'VENDOR_ADDRESS_COUNTRY_NAME', 'EDUCATIONAL_INSTITUTION_FLAG', 
        'FIRM_8A_FLAG', 'WOMEN_OWNED_FLAG', 
        'WOSB_FLAG', 'JVWOSB_FLAG', 'EDWOSB_FLAG', 'EDJVWOSB_FLAG', 
        'FIRM8A_JOINT_VENTURE', 'FEDERALLY_FUNDED_R_AND_D_CORP', 'CORP_ENTITY_NOT_TAX_EXEMPT', 'CORP_ENTITY_TAX_EXEMPT', 
        'PARTNERSHIP_OR_LLP', 'SOLE_PROPREITORSHIP', 'SMALL_AGRICULTURAL_COOPERATIVE', 'INTERNATIONAL_ORGANIZATION', 
        'ARCHITECTURE_AND_ENGINEERING', 'COMMUNITY_CORP_OWNED_FIRM', 'CONSTRUCTION_FIRM', 'DOMESTIC_SHELTER', 
        'FOUNDATION', 'MANUFACTURER_OF_GOODS', 'RESEARCH_AND_DEVELOPMENT', 'SERVICE_PROVIDER', 
        'VETERINARY_HOSPITAL', 'HISPANIC_SERVICING_INSTITUTION', 'LIMITED_LIABILITY_CORPORATION',
        'AWARD_OR_IDV','CO_BUS_SIZE_DETERMINATION','PRINCIPAL_NAICS_CODE','PRODUCT_OR_SERVICE_CODE', #karolina
        'FUNDING_DEPARTMENT_ID','PIID','IDV_PIID','MODIFICATION_NUMBER', #for unique ID
       'IDV_CONTRACTING_AGENCY_NAME', 'IDV_EXTENT_COMPETED', 'IDV_EVALUATED_PREFERENCE', 'PART8_OR_PART13', #kevin
       'PRODUCT_OR_SERVICE_TYPE', 
       'FOR_PROFIT_ORGANIZATION', 'DOT_CERTIFIED_DISADV_BUS', 'SDB', 'DEPARTMENT', 'AGENCY', #david
       'FISCAL_YEAR']

In [15]:
%%time
test_2022 = pl.read_parquet('2022.parquet', columns=cols)

CPU times: user 7.33 s, sys: 17.2 s, total: 24.5 s
Wall time: 10 s


In [12]:
test_2022 = test_2022.filter(test_2022['CO_BUS_SIZE_DETERMINATION']=="SMALL BUSINESS")
test_2022.shape

(3167065, 45)

In [16]:
def replace_yes_no_values(column):
    return column.map_elements(lambda x: 1 if x == 'YES' else (0 if x == 'NO' else x))

flag_columns = ['EDUCATIONAL_INSTITUTION_FLAG', 'FIRM_8A_FLAG', 'WOMEN_OWNED_FLAG',
                'WOSB_FLAG', 'JVWOSB_FLAG', 'EDWOSB_FLAG', 'EDJVWOSB_FLAG',
                'FIRM8A_JOINT_VENTURE', 'FEDERALLY_FUNDED_R_AND_D_CORP',
                'CORP_ENTITY_NOT_TAX_EXEMPT', 'CORP_ENTITY_TAX_EXEMPT',
                'PARTNERSHIP_OR_LLP', 'SOLE_PROPREITORSHIP', 'SMALL_AGRICULTURAL_COOPERATIVE',
                'INTERNATIONAL_ORGANIZATION', 'ARCHITECTURE_AND_ENGINEERING',
                'COMMUNITY_CORP_OWNED_FIRM', 'CONSTRUCTION_FIRM', 'DOMESTIC_SHELTER',
                'FOUNDATION', 'MANUFACTURER_OF_GOODS', 'RESEARCH_AND_DEVELOPMENT',
                'SERVICE_PROVIDER', 'VETERINARY_HOSPITAL', 'HISPANIC_SERVICING_INSTITUTION',
                'LIMITED_LIABILITY_CORPORATION']

for col in flag_columns:
    test_2022 = test_2022.with_columns(replace_yes_no_values(test_2022[col]))

In [17]:
#combine women-owned columns into one
women_summed = test_2022.select(['WOMEN_OWNED_FLAG', 'WOSB_FLAG', 'JVWOSB_FLAG', 'EDWOSB_FLAG', 'EDJVWOSB_FLAG']).transpose().sum().transpose()
test_2022 = test_2022.with_columns(women_summed["column_0"].map_elements(lambda x: 1 if x > 0 else 0).alias("WOMEN_OWNED_FLAG"))
test_2022 = test_2022.drop(['WOSB_FLAG', 'JVWOSB_FLAG', 'EDWOSB_FLAG', 'EDJVWOSB_FLAG'])
test_2022.shape

(6088454, 44)